<a href="https://colab.research.google.com/github/Shibu778/MyNotebooks/blob/master/Extracted_Features_with_constant_time_delay_and_embedding_dimension.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook consists of code to extract the area under the amplification factor histogram from Anil Verma and Vinod Grover Dataset. The time delay and embedding dimension are taken constant.

# Initial Setup


- Step 1 is to mount the drive.
- Step 2 is to change the current working directory to the My Drive Folder where the Nolitsa package is there.
- Step 3 is to import the required library.

In [ ]:
cd drive/My\ Drive

/content/drive/My Drive


In [ ]:
# Importing the required library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nolitsa.nolitsa import lyapunov, utils, delay, dimension
from scipy.spatial import cKDTree as KDTree
import os

- Define some useful functions

In [ ]:
# Utility Functions
def txt_to_arr(filepath_in):
    '''
    Function : Takes the relative file path and read the *.txt file mentioned in the path and
    gives an numpy array as output.
    Input : filepath
    Output : Numpy array containing the single column signal inside the txt file
    Use the following file format
    filepath = '/content/drive/My Drive/Sleep data/Amit Agrahari/C3 - 14.04.2014.txt'
    txt_to_mat(filepath)
    '''
    df = pd.read_csv(filepath_in,header=None,encoding= 'unicode_escape')
    df = df[6:]
    arr = np.squeeze(df.to_numpy(),axis = 1)
    arr = arr.astype(float)
    print("Length of data : ", len(arr))
    return arr
def read_sleep_profile(filename):
  profile = []
  with open(filename) as f:
    for row in f:
      profile.append(row.split(';'))

  profile = profile[7:]
  pr = []
  for i in range(len(profile)):
    pr.append(profile[i][1][:-1])

  pr = np.asarray(pr)
  return pr

def find_area(x,x1, dim = [6], tau = 10):
  #r = delay.acorr(x, maxtau=50)
  #tau = np.argmax(r < 1.0 / np.e)
  data1 = np.asarray([utils.reconstruct(x, dim=d, tau=tau) for d in dim], dtype = float)
  data1 = np.squeeze(data1)
  data2 = np.asarray([utils.reconstruct(x1, dim=d, tau=tau) for d in dim], dtype = float)
  data2 = np.squeeze(data2)
  tree = KDTree(data1)
  j = 1 # index of the point whose nearest neighbor is taken into study
  for i in range(2, 23):
    dist, index = tree.query(data1[j], k = i, p =2) # p=2 for euclidean distance

  # So point with index 1 and 4477 can be taken as the two point in closest trajectories 
  #(Considering the Theiler wondow; as 4477 is greater than the theiler window)
  # Finding the amplification upto next 1000 points in each
  A = np.array([],dtype = float)
  d0 = dist[1]
  A = np.append(A,d0)
  for i in range(1,3000):
    d1 = np.asarray(euclidean_distance(data2[j+i],data2[index[1]+i]), dtype = float)
    A = np.append(A,d1)

  # Plotting the histogram
  plt.figure(2)
  n, bins = np.histogram(A, bins=100)
  area = sum(np.diff(bins)*n)
  return area

def euclidean_distance(x,y):
  return np.sqrt((x[0]-y[0])**2+(x[1]-y[1])**2+(x[2]-y[2])**2)

def best_fit_slope(xs,ys):
    m = (((np.mean(xs)*np.mean(ys)) - np.mean(xs*ys)) /
         ((np.mean(xs)**2) - np.mean(xs**2)))
    return m


- Initialize the time delay and embedding dimension value to some particular value


In [ ]:
tau = 20
dim = 5

# Extracting Features from Vinod Grover Data

- Change directory to Vinod Grover directory

In [ ]:
cd Sleep data/Vinod Grover

/content/drive/My Drive/Sleep data/Vinod Grover


- Read the C3, O2, F3 and ECG channel from a csv file

In [ ]:
df = pd.read_csv("C3_O2_F3_ECG_Vinod_Grover.csv")
df.head()

,C3,O2,F3,ECG
0,-0.012207,-0.012207,-0.012207,0.036622
1,-0.012207,-0.012207,-0.012207,0.036622
2,-0.012207,-0.012207,-0.012207,0.036622
3,-0.012207,-0.012207,-0.012207,0.036622
4,-0.012207,-0.012207,-0.012207,0.036622


- Extract each channels to a python variable and see the length

In [ ]:
C3 = df.C3.values
O2 = df.O2.values
F3 = df.F3.values
ECG = df.ECG.values
len(C3), len(O2), len(F3), len(ECG)

(7480064, 7480064, 7480064, 7480064)

## Reading the Sleep profile

In [ ]:
# Reading the sleep profile
filename = 'Sleep profile.txt'
profile = read_sleep_profile(filename)
len(profile)

975

In [ ]:
profile[:5]

array([' A', ' Wake', ' Wake', ' Wake', ' Wake'], dtype='<U5')

In [ ]:
set(profile)

{' A', ' N1', ' N2', ' N3', ' N4', ' REM', ' Wake'}

In [ ]:
[np.sum(profile == ' A')/len(profile), np.sum(profile == ' N1')/len(profile),np.sum(profile == ' N2')/len(profile),
np.sum(profile == ' N3')/len(profile), np.sum(profile == ' N4')/len(profile), np.sum(profile == ' REM')/len(profile),
np.sum(profile == ' Wake')/len(profile)]

[0.0020512820512820513,
 0.32,
 0.27487179487179486,
 0.021538461538461538,
 0.005128205128205128,
 0.024615384615384615,
 0.3517948717948718]

In [ ]:
profile[profile == ' A'] = 0
profile[profile == ' N1'] = 1
profile[profile == ' N2'] = 2
profile[profile == ' N3'] = 3
profile[profile == ' N4'] = 4
profile[profile == ' REM'] = 5
profile[profile == ' Wake'] = 6
profile

In [ ]:
len(profile)

975

## Extracting Features from C3 channel and storing it in a dataframe ar_label


In [ ]:
C3 = C3/max(C3)
area_C3 = np.array([], dtype = float)
for i in range(int(len(C3)/7680)-1):
  #print(i)
  x = C3[i*7680:i*7680+7680]
  x1 = C3[i*7680:i*7680+2*7680]
  #print(x)
  ar = np.asarray([find_area(x,x1, dim = [dim], tau = tau)],dtype=float)
  area_C3 = np.append(area_C3,ar)
print(len(area_C3))
ar_label = pd.DataFrame({'area_C3':area_C3})
ar_label.head()

972


,area_C3
0,30.000000
1,3.010115
2,2.448438
3,1.399680
4,0.310114


<Figure size 432x288 with 0 Axes>

## Extracting Features from O2 channel and storing it in ar_label

In [ ]:
O2 = O2/max(O2)
area_O2 = np.array([], dtype = float)
for i in range(int(len(O2)/7680)-1):
  #print(i)
  x = O2[i*7680:i*7680+7680]
  x1 = O2[i*7680:i*7680+2*7680]
  #print(x)
  ar = np.asarray([find_area(x,x1, dim = [dim], tau = tau)],dtype=float)
  area_O2 = np.append(area_O2,ar)
print(len(area_O2))
ar_label['area_O2'] = area_O2
ar_label.head()

972


,area_C3,area_O2
0,30.000000,30.000000
1,3.010115,16.586917
2,2.448438,0.349703
3,1.399680,2.129442
4,0.310114,2.099614


<Figure size 432x288 with 0 Axes>

## Extracting Features from F3 channel

In [ ]:
F3 = F3/max(F3)
area_F3 = np.array([], dtype = float)
for i in range(int(len(F3)/7680)-1):
  #print(i)
  x = F3[i*7680:i*7680+7680]
  x1 = F3[i*7680:i*7680+2*7680]
  #print(x)
  ar = np.asarray([find_area(x,x1, dim = [dim], tau = tau)],dtype=float)
  area_F3 = np.append(area_F3,ar)
print(len(area_F3))
ar_label['area_F3'] = area_F3
ar_label.head()

972


,area_C3,area_O2,area_F3
0,30.000000,30.000000,30.000000
1,3.010115,16.586917,3.570716
2,2.448438,0.349703,3.588676
3,1.399680,2.129442,0.243728
4,0.310114,2.099614,2.517491


<Figure size 432x288 with 0 Axes>

## Extracting Features from ECG channel


In [ ]:
ECG = ECG/max(ECG)
area_ECG = np.array([], dtype = float)
for i in range(int(len(ECG)/7680)-1):
  #print(i)
  x = ECG[i*7680:i*7680+7680]
  x1 = ECG[i*7680:i*7680+2*7680]
  #print(x)
  ar = np.asarray([find_area(x,x1, dim = [dim], tau = tau)],dtype=float)
  area_ECG = np.append(area_ECG,ar)
print(len(area_ECG))
ar_label['area_ECG'] = area_ECG
ar_label.head()

972


,area_C3,area_O2,area_F3,area_ECG
0,30.000000,30.000000,30.000000,30.000000
1,3.010115,16.586917,3.570716,11.843110
2,2.448438,0.349703,3.588676,10.811324
3,1.399680,2.129442,0.243728,6.099449
4,0.310114,2.099614,2.517491,11.527038


<Figure size 432x288 with 0 Axes>

- Adding the profile column

In [ ]:
len(profile[:-3])

972

In [ ]:
# Adding the label
ar_label['label'] = profile[:-3]
ar_label.head()

,area_C3,area_O2,area_F3,area_ECG,label
0,30.000000,30.000000,30.000000,30.000000,0
1,3.010115,16.586917,3.570716,11.843110,6
2,2.448438,0.349703,3.588676,10.811324,6
3,1.399680,2.129442,0.243728,6.099449,6
4,0.310114,2.099614,2.517491,11.527038,6


In [ ]:
ar_label.to_csv('Feature_C3_O2_F3_label_Vinod_constant_tau_dim.csv', index = False)

In [ ]:
df2 = pd.read_csv('Feature_C3_O2_F3_label_Vinod_constant_tau_dim.csv')
df2

,area_C3,area_O2,area_F3,area_ECG,label
0,30.000000,30.000000,30.000000,30.000000,0
1,3.010115,16.586917,3.570716,11.843110,6
2,2.448438,0.349703,3.588676,10.811324,6
3,1.399680,2.129442,0.243728,6.099449,6
4,0.310114,2.099614,2.517491,11.527038,6
...,...,...,...,...,...
967,0.188498,1.538036,0.205539,16.169665,1
968,1.025179,1.637132,0.201827,17.702024,6
969,0.760965,1.876240,1.507760,16.094735,1
970,0.860991,0.306512,1.091936,7.705208,1


# Extracting Features from Anil Verma Dataset

In [ ]:
cd ../Anil\ Verma

/content/drive/My Drive/Sleep data/Anil Verma


In [ ]:
df1 = pd.read_csv("Anil_Verma_C3_O2_F3.csv")
df2 = pd.read_csv("Anil_Verma_ECG.csv")
df1.head(),df2.head()

(         C3        O2        F3
 0 -0.012207 -0.012207 -0.012207
 1 -0.012207 -0.012207 -0.012207
 2 -0.012207 -0.012207 -0.012207
 3 -0.012207 -0.012207 -0.012207
 4 -0.012207 -0.012207 -0.012207,         ECG
 0  0.036622
 1  0.036622
 2  0.036622
 3  0.036622
 4  0.036622)

In [ ]:
C3 = df1.C3.values
O2 = df1.O2.values
F3 = df1.F3.values
ECG = df2.ECG.values
len(C3), len(O2), len(F3), len(ECG)

(6344704, 6344704, 6344704, 6344704)

## Reading the Sleep profile


In [ ]:
# Reading the sleep profile
filename = 'Sleep profile.txt'
profile = read_sleep_profile(filename)
len(profile)

827

In [ ]:
profile[:5]

array([' A', ' Wake', ' Wake', ' Wake', ' Wake'], dtype='<U9')

In [ ]:
set(profile)

{' A', ' Artefact', ' N1', ' N2', ' N3', ' REM', ' Wake'}

In [ ]:
[np.sum(profile == ' Artefact')/len(profile), np.sum(profile == ' N1')/len(profile),np.sum(profile == ' N2')/len(profile),
np.sum(profile == ' N3')/len(profile), np.sum(profile == ' N4')/len(profile), np.sum(profile == ' REM')/len(profile),
np.sum(profile == ' Wake')/len(profile)]

[0.013301088270858524,
 0.010882708585247884,
 0.2370012091898428,
 0.3567110036275695,
 0.0,
 0.26844014510278114,
 0.1124546553808948]

In [ ]:
profile[profile == ' Artefact'] = 0
profile[profile == ' A'] = 0
profile[profile == ' N1'] = 1
profile[profile == ' N2'] = 2
profile[profile == ' N3'] = 3
profile[profile == ' N4'] = 4
profile[profile == ' REM'] = 5
profile[profile == ' Wake'] = 6
profile

In [ ]:
len(profile)

827

## Extracting and storing Features from C3 channel

In [ ]:
C3 = C3/max(C3)
area_C3 = np.array([], dtype = float)
for i in range(int(len(C3)/7680)-1):
  #print(i)
  x = C3[i*7680:i*7680+7680]
  x1 = C3[i*7680:i*7680+2*7680]
  #print(x)
  ar = np.asarray([find_area(x,x1, dim = [dim], tau = tau)],dtype=float)
  area_C3 = np.append(area_C3,ar)
print(len(area_C3))
ar_label = pd.DataFrame({'area_C3':area_C3})
ar_label.head()

825


,area_C3
0,1.106912
1,2.793432
2,9.179584
3,3.146348
4,2.560994


<Figure size 432x288 with 0 Axes>

## Extracting and storing Features from O2 channel

In [ ]:
O2 = O2/max(O2)
area_O2 = np.array([], dtype = float)
for i in range(int(len(O2)/7680)-1):
  #print(i)
  x = O2[i*7680:i*7680+7680]
  x1 = O2[i*7680:i*7680+2*7680]
  #print(x)
  ar = np.asarray([find_area(x,x1, dim = [dim], tau = tau)],dtype=float)
  area_O2 = np.append(area_O2,ar)
print(len(area_O2))
ar_label['area_O2'] = area_O2
ar_label.head()

825


,area_C3,area_O2
0,1.106912,1.136696
1,2.793432,2.842188
2,9.179584,7.207367
3,3.146348,3.121649
4,2.560994,2.577528


<Figure size 432x288 with 0 Axes>

## Extracting and storing Features from F3 channel

In [ ]:
F3 = F3/max(F3)
area_F3 = np.array([], dtype = float)
for i in range(int(len(F3)/7680)-1):
  #print(i)
  x = F3[i*7680:i*7680+7680]
  x1 = F3[i*7680:i*7680+2*7680]
  #print(x)
  ar = np.asarray([find_area(x,x1, dim = [dim], tau = tau)],dtype=float)
  area_F3 = np.append(area_F3,ar)
print(len(area_F3))
ar_label['area_F3'] = area_F3
ar_label.head()

825


,area_C3,area_O2,area_F3
0,1.106912,1.136696,1.146486
1,2.793432,2.842188,2.638771
2,9.179584,7.207367,8.277753
3,3.146348,3.121649,3.579870
4,2.560994,2.577528,0.647855


<Figure size 432x288 with 0 Axes>

## Extracting and storing Features from ECG channel

In [ ]:
ECG = ECG/max(ECG)
area_ECG = np.array([], dtype = float)
for i in range(int(len(ECG)/7680)-1):
  #print(i)
  x = ECG[i*7680:i*7680+7680]
  x1 = ECG[i*7680:i*7680+2*7680]
  #print(x)
  ar = np.asarray([find_area(x,x1, dim = [dim], tau = tau)],dtype=float)
  area_ECG = np.append(area_ECG,ar)
print(len(area_ECG))
ar_label['area_ECG'] = area_ECG
ar_label.head()

825


,area_C3,area_O2,area_F3,area_ECG
0,1.106912,1.136696,1.146486,8.091134
1,2.793432,2.842188,2.638771,28.077340
2,9.179584,7.207367,8.277753,31.128761
3,3.146348,3.121649,3.579870,7.836355
4,2.560994,2.577528,0.647855,28.523173


<Figure size 432x288 with 0 Axes>

- Adding the profile column

In [ ]:
len(profile[:-2])

825

In [ ]:
# Adding the label
ar_label['label'] = profile[:-2]
ar_label.head()

,area_C3,area_O2,area_F3,area_ECG,label
0,1.106912,1.136696,1.146486,8.091134,0
1,2.793432,2.842188,2.638771,28.077340,6
2,9.179584,7.207367,8.277753,31.128761,6
3,3.146348,3.121649,3.579870,7.836355,6
4,2.560994,2.577528,0.647855,28.523173,6


In [ ]:
ar_label.to_csv('Feature_C3_O2_F3_label_Anil_constant_tau_dim.csv', index = False)

In [ ]:
df2 = pd.read_csv('Feature_C3_O2_F3_label_Anil_constant_tau_dim.csv')
df2

,area_C3,area_O2,area_F3,area_ECG,label
0,1.106912,1.136696,1.146486,8.091134,0
1,2.793432,2.842188,2.638771,28.077340,6
2,9.179584,7.207367,8.277753,31.128761,6
3,3.146348,3.121649,3.579870,7.836355,6
4,2.560994,2.577528,0.647855,28.523173,6
...,...,...,...,...,...
820,0.586899,0.596908,0.624382,28.149301,5
821,4.571104,4.732546,4.731905,8.132409,6
822,1.246982,1.274810,1.286997,9.320826,6
823,11.760017,12.098599,7.792394,33.078858,6


# Extracting Features from Amit Agrahari Dataset

In [ ]:
cd ../Amit\ Agrahari

/content/drive/My Drive/Sleep data/Amit Agrahari


In [ ]:
ls

 Amit_Agrahari_C3_F3_O2.csv  'Data for sir'/          F3.mat
 Amit_Agrahari_ECG.csv       'ECG - 14.04.2014.txt'  'O1 - 14.04.2014.txt'
'C3 - 14.04.2014.txt'         ECG.mat                 O1.mat
 C3.mat                      'F3 - 14.04.2014.txt'   'O2 - 14.04.2014.txt'


## Storing data into csv file

In [ ]:
C3 = txt_to_arr('C3 - 14.04.2014.txt')
ECG = txt_to_arr('ECG - 14.04.2014.txt')
F3 = txt_to_arr('F3 - 14.04.2014.txt')
O2 = txt_to_arr('O2 - 14.04.2014.txt')
len(C3), len(F3), len(O2), len(ECG)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Length of data :  4700161
Length of data :  9400321
Length of data :  4700161
Length of data :  4700161


(4700161, 4700161, 4700161, 9400321)

In [ ]:
df = pd.DataFrame()
df['C3'] = C3
df['F3'] = F3
df['O2'] = O2
df.head()

,C3,F3,O2
0,25.0,0.0,0.0
1,21.0,0.0,0.0
2,19.0,0.0,0.0
3,16.0,0.0,0.0
4,14.0,0.0,0.0


In [ ]:
df.to_csv('Amit_Agrahari_C3_F3_O2.csv', index = None)

In [ ]:
df1 = pd.DataFrame()
df1['ECG'] = ECG
df1.to_csv('Amit_Agrahari_ECG.csv', index = None)
df1.head()

,ECG
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


## Reading the csv file

In [ ]:
df = pd.read_csv('Amit_Agrahari_C3_F3_O2.csv')
df.head()

,C3,F3,O2
0,25.0,0.0,0.0
1,21.0,0.0,0.0
2,19.0,0.0,0.0
3,16.0,0.0,0.0
4,14.0,0.0,0.0


In [ ]:
df1 = pd.read_csv('Amit_Agrahari_ECG.csv')
df1.head()

,ECG
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


In [ ]:
C3 = df.C3.values
F3 = df.F3.values
O2 = df.O2.values
ECG = df1.ECG.values
ECG = ECG[range(1, len(ECG), 2)]
len(C3), len(F3), len(O2), len(ECG)

(4700161, 4700161, 4700161, 4700160)

## No sleep profile available

## Extracting and storing Features from C3 channel

In [ ]:
C3 = C3/max(C3)
area_C3 = np.array([], dtype = float)
for i in range(int(len(C3)/7680)-1):
  #print(i)
  x = C3[i*7680:i*7680+7680]
  x1 = C3[i*7680:i*7680+2*7680]
  #print(x)
  ar = np.asarray([find_area(x,x1, dim = [dim], tau = tau)],dtype=float)
  area_C3 = np.append(area_C3,ar)
print(len(area_C3))
ar_label = pd.DataFrame({'area_C3':area_C3})
ar_label.head()

611


,area_C3
0,0.112500
1,1.747275
2,1.691571
3,1.846852
4,19.962917


<Figure size 432x288 with 0 Axes>

## Extracting and storing Features from F3 channel

In [ ]:
F3 = F3/max(F3)
area_F3 = np.array([], dtype = float)
for i in range(int(len(F3)/7680)-1):
  #print(i)
  x = F3[i*7680:i*7680+7680]
  x1 = F3[i*7680:i*7680+2*7680]
  #print(x)
  ar = np.asarray([find_area(x,x1, dim = [dim], tau = tau)],dtype=float)
  area_F3 = np.append(area_F3,ar)
print(len(area_F3))
ar_label['area_F3'] = area_F3
ar_label.head()

611


,area_C3,area_F3
0,0.112500,30.000000
1,1.747275,1.840806
2,1.691571,2.066848
3,1.846852,1.985464
4,19.962917,20.764247


<Figure size 432x288 with 0 Axes>

## Extracting and storing featues from O2 channel

In [ ]:
O2 = O2/max(O2)
area_O2 = np.array([], dtype = float)
for i in range(int(len(O2)/7680)-1):
  #print(i)
  x = O2[i*7680:i*7680+7680]
  x1 = O2[i*7680:i*7680+2*7680]
  #print(x)
  ar = np.asarray([find_area(x,x1, dim = [dim], tau = tau)],dtype=float)
  area_O2 = np.append(area_O2,ar)
print(len(area_O2))
ar_label['area_O2'] = area_O2
ar_label.head()

611


,area_C3,area_F3,area_O2
0,0.112500,30.000000,30.000000
1,1.747275,1.840806,1.609279
2,1.691571,2.066848,1.676696
3,1.846852,1.985464,1.720606
4,19.962917,20.764247,20.542833


<Figure size 432x288 with 0 Axes>

## Extracting and Storing Features from ECG

In [ ]:
ECG = ECG/max(ECG)
area_ECG = np.array([], dtype = float)
for i in range(int(len(ECG)/7680)-1):
  #print(i)
  x = ECG[i*7680:i*7680+7680]
  x1 = ECG[i*7680:i*7680+2*7680]
  #print(x)
  ar = np.asarray([find_area(x,x1, dim = [dim], tau = tau)],dtype=float)
  area_ECG = np.append(area_ECG,ar)
print(len(area_ECG))
ar_label['area_ECG'] = area_ECG
ar_label.head()

611


,area_C3,area_F3,area_O2,area_ECG
0,0.112500,30.000000,30.000000,30.000000
1,1.747275,1.840806,1.609279,45.944219
2,1.691571,2.066848,1.676696,45.588827
3,1.846852,1.985464,1.720606,29.785646
4,19.962917,20.764247,20.542833,43.572935


<Figure size 432x288 with 0 Axes>

In [ ]:
ar_label.to_csv('Feature_C3_F3_O2_ECG_nolabel_Amit_constant_tau_dim.csv', index = False)

In [ ]:
df = pd.read_csv('Feature_C3_F3_O2_ECG_nolabel_Amit_constant_tau_dim.csv')
df.head()

,area_C3,area_F3,area_O2,area_ECG
0,0.112500,30.000000,30.000000,30.000000
1,1.747275,1.840806,1.609279,45.944219
2,1.691571,2.066848,1.676696,45.588827
3,1.846852,1.985464,1.720606,29.785646
4,19.962917,20.764247,20.542833,43.572935


In [ ]:
len(df)

611

# Extracting Features from Arnavi Gupta

In [ ]:
cd ../Arnavi\ Gupta

/content/drive/My Drive/Sleep data/Arnavi Gupta


In [ ]:
ls

'C3 - 03.02.2016.txt'     'F3 - 03.02.2016.txt'
'ECG 2 - 03.02.2016.txt'  'O2 - 03.02.2016.txt'


## Storing the Data in csv file

In [ ]:
C3 = txt_to_arr('C3 - 03.02.2016.txt')
F3 = txt_to_arr('F3 - 03.02.2016.txt')
O2 = txt_to_arr('O2 - 03.02.2016.txt')
ECG = txt_to_arr('ECG 2 - 03.02.2016.txt')
ECG = ECG[range(0, len(ECG), 2)]
len(C3), len(F3), len(O2), len(ECG)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Length of data :  2311168
Length of data :  2311168
Length of data :  2311168
Length of data :  4622336


(2311168, 2311168, 2311168, 2311168)

In [ ]:
df = pd.DataFrame()
df['C3'] = C3
df['F3'] = F3
df['O2'] = O2
df['ECG'] = ECG
df.to_csv('Arnavi_Gupta_C3_F3_O2_ECG.csv', index = None)
df.head()

,C3,F3,O2,ECG
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0


## Reading the csv file

In [ ]:
df = pd.read_csv('Arnavi_Gupta_C3_F3_O2_ECG.csv')
df.head()

,C3,F3,O2,ECG
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0


In [ ]:
C3 = df.C3.values
F3 = df.F3.values
O2 = df.O2.values
ECG = df.ECG.values
len(C3), len(F3), len(O2), len(ECG)

(2311168, 2311168, 2311168, 2311168)

## Extracting and Storing Features from C3 Channel

In [ ]:
C3 = C3/max(C3)
area_C3 = np.array([], dtype = float)
for i in range(int(len(C3)/7680)-1):
  #print(i)
  x = C3[i*7680:i*7680+7680]
  x1 = C3[i*7680:i*7680+2*7680]
  #print(x)
  ar = np.asarray([find_area(x,x1, dim = [dim], tau = tau)],dtype=float)
  area_C3 = np.append(area_C3,ar)
print(len(area_C3))
ar_label = pd.DataFrame({'area_C3':area_C3})
ar_label.head()

299


,area_C3
0,6.210094
1,3.230525
2,3.821619
3,0.842749
4,0.841668


<Figure size 432x288 with 0 Axes>

## Extracting and Storing Features from F3 channel

In [ ]:
F3 = F3/max(F3)
area_F3 = np.array([], dtype = float)
for i in range(int(len(F3)/7680)-1):
  #print(i)
  x = F3[i*7680:i*7680+7680]
  x1 = F3[i*7680:i*7680+2*7680]
  #print(x)
  ar = np.asarray([find_area(x,x1, dim = [dim], tau = tau)],dtype=float)
  area_F3 = np.append(area_F3,ar)
print(len(area_F3))
ar_label['area_F3'] = area_F3
ar_label.head()

299


,area_C3,area_F3
0,6.210094,1.433811
1,3.230525,0.494932
2,3.821619,0.576153
3,0.842749,0.534944
4,0.841668,0.494932


<Figure size 432x288 with 0 Axes>

## Extracting and Storing Features from O2 channel

In [ ]:
O2 = O2/max(O2)
area_O2 = np.array([], dtype = float)
for i in range(int(len(O2)/7680)-1):
  #print(i)
  x = O2[i*7680:i*7680+7680]
  x1 = O2[i*7680:i*7680+2*7680]
  #print(x)
  ar = np.asarray([find_area(x,x1, dim = [dim], tau = tau)],dtype=float)
  area_O2 = np.append(area_O2,ar)
print(len(area_O2))
ar_label['area_O2'] = area_O2
ar_label.head()

299


,area_C3,area_F3,area_O2
0,6.210094,1.433811,7.966651
1,3.230525,0.494932,7.379974
2,3.821619,0.576153,1.484779
3,0.842749,0.534944,1.688526
4,0.841668,0.494932,1.879593


<Figure size 432x288 with 0 Axes>

## Extracting and Storing Features from ECG channel

In [ ]:
ECG = ECG/max(ECG)
area_ECG = np.array([], dtype = float)
for i in range(int(len(ECG)/7680)-1):
  #print(i)
  x = ECG[i*7680:i*7680+7680]
  x1 = ECG[i*7680:i*7680+2*7680]
  #print(x)
  ar = np.asarray([find_area(x,x1, dim = [dim], tau = tau)],dtype=float)
  area_ECG = np.append(area_ECG,ar)
print(len(area_ECG))
ar_label['area_ECG'] = area_ECG
ar_label.head()

299


,area_C3,area_F3,area_O2,area_ECG
0,6.210094,1.433811,7.966651,31.348936
1,3.230525,0.494932,7.379974,29.983543
2,3.821619,0.576153,1.484779,30.955552
3,0.842749,0.534944,1.688526,41.580560
4,0.841668,0.494932,1.879593,16.862991


<Figure size 432x288 with 0 Axes>

In [ ]:
ar_label.to_csv('Feature_C3_F3_O2_ECG_nolabel_Arnavi_constant_tau_dim.csv', index = False)
df = pd.read_csv('Feature_C3_F3_O2_ECG_nolabel_Arnavi_constant_tau_dim.csv')
df.head()

,area_C3,area_F3,area_O2,area_ECG
0,6.210094,1.433811,7.966651,31.348936
1,3.230525,0.494932,7.379974,29.983543
2,3.821619,0.576153,1.484779,30.955552
3,0.842749,0.534944,1.688526,41.580560
4,0.841668,0.494932,1.879593,16.862991


# Extracting Features from Ashi

In [ ]:
cd ../Ashi\ Jaini

/content/drive/My Drive/Sleep data/Ashi Jaini


In [ ]:
ls

'C3 - 10.07.2019.txt'   'F3 - 10.07.2019.txt'
'ECG - 10.07.2019.txt'  'O2 - 10.07.2019.txt'


## Storing data into csv file

In [ ]:
C3 = txt_to_arr('C3 - 10.07.2019.txt')
F3 = txt_to_arr('F3 - 10.07.2019.txt')
O2 = txt_to_arr('O2 - 10.07.2019.txt')
ECG = txt_to_arr('ECG - 10.07.2019.txt')
ECG = ECG[range(0, len(ECG), 2)]
len(C3), len(F3), len(O2), len(ECG)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Length of data :  2012160
Length of data :  2012160
Length of data :  2012160
Length of data :  4024320


(2012160, 2012160, 2012160, 2012160)

In [ ]:
df = pd.DataFrame()
df['C3'] = C3
df['F3'] = F3
df['O2'] = O2
df['ECG'] = ECG
df.to_csv('Ashi_Jaini_C3_F3_O2_ECG.csv', index = None)
df.head()

,C3,F3,O2,ECG
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0


## Reading the csv file

In [ ]:
df = pd.read_csv('Ashi_Jaini_C3_F3_O2_ECG.csv')
df.head()

,C3,F3,O2,ECG
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0


In [ ]:
C3 = df.C3.values
F3 = df.F3.values
O2 = df.O2.values
ECG = df.ECG.values
len(C3), len(F3), len(O2), len(ECG)

(2012160, 2012160, 2012160, 2012160)

## Extracting and Storing Features from C3 channel

In [ ]:
C3 = C3/max(C3)
area_C3 = np.array([], dtype = float)
for i in range(int(len(C3)/7680)-1):
  #print(i)
  x = C3[i*7680:i*7680+7680]
  x1 = C3[i*7680:i*7680+2*7680]
  #print(x)
  ar = np.asarray([find_area(x,x1, dim = [dim], tau = tau)],dtype=float)
  area_C3 = np.append(area_C3,ar)
print(len(area_C3))
ar_label = pd.DataFrame({'area_C3':area_C3})
ar_label.head()

261


,area_C3
0,30.000000
1,30.000000
2,2.468172
3,5.235664
4,5.148358


<Figure size 432x288 with 0 Axes>

## Extracting and Storing Features from F3 channel

In [ ]:
F3 = F3/max(F3)
area_F3 = np.array([], dtype = float)
for i in range(int(len(F3)/7680)-1):
  #print(i)
  x = F3[i*7680:i*7680+7680]
  x1 = F3[i*7680:i*7680+2*7680]
  #print(x)
  ar = np.asarray([find_area(x,x1, dim = [dim], tau = tau)],dtype=float)
  area_F3 = np.append(area_F3,ar)
print(len(area_F3))
ar_label['area_F3'] = area_F3
ar_label.head()

261


,area_C3,area_F3
0,30.000000,30.000000
1,30.000000,30.000000
2,2.468172,2.583993
3,5.235664,6.076265
4,5.148358,4.871707


<Figure size 432x288 with 0 Axes>

## Extracting and Storing Features from O2 channel

In [ ]:
O2 = O2/max(O2)
area_O2 = np.array([], dtype = float)
for i in range(int(len(O2)/7680)-1):
  #print(i)
  x = O2[i*7680:i*7680+7680]
  x1 = O2[i*7680:i*7680+2*7680]
  #print(x)
  ar = np.asarray([find_area(x,x1, dim = [dim], tau = tau)],dtype=float)
  area_O2 = np.append(area_O2,ar)
print(len(area_O2))
ar_label['area_O2'] = area_O2
ar_label.head()

261


,area_C3,area_F3,area_O2
0,30.000000,30.000000,30.000000
1,30.000000,30.000000,30.000000
2,2.468172,2.583993,2.839266
3,5.235664,6.076265,8.621127
4,5.148358,4.871707,4.835506


<Figure size 432x288 with 0 Axes>

## Extracting and Storing Features from ECG channel

In [ ]:
ECG = ECG/max(ECG)
area_ECG = np.array([], dtype = float)
for i in range(int(len(ECG)/7680)-1):
  #print(i)
  x = ECG[i*7680:i*7680+7680]
  x1 = ECG[i*7680:i*7680+2*7680]
  #print(x)
  ar = np.asarray([find_area(x,x1, dim = [dim], tau = tau)],dtype=float)
  area_ECG = np.append(area_ECG,ar)
print(len(area_ECG))
ar_label['area_ECG'] = area_ECG
ar_label.head()

261


,area_C3,area_F3,area_O2,area_ECG
0,30.000000,30.000000,30.000000,30.000000
1,30.000000,30.000000,30.000000,30.000000
2,2.468172,2.583993,2.839266,18.663955
3,5.235664,6.076265,8.621127,19.340948
4,5.148358,4.871707,4.835506,16.822310


<Figure size 432x288 with 0 Axes>

In [ ]:
ar_label.to_csv('Feature_C3_F3_O2_ECG_nolabel_Ashi_constant_tau_dim.csv', index = False)
df = pd.read_csv('Feature_C3_F3_O2_ECG_nolabel_Ashi_constant_tau_dim.csv')
df.head()

,area_C3,area_F3,area_O2,area_ECG
0,30.000000,30.000000,30.000000,30.000000
1,30.000000,30.000000,30.000000,30.000000
2,2.468172,2.583993,2.839266,18.663955
3,5.235664,6.076265,8.621127,19.340948
4,5.148358,4.871707,4.835506,16.822310


# Extracting Features from Durgesh

In [ ]:
cd ../Durgesh\ Singh

/content/drive/My Drive/Sleep data/Durgesh Singh


In [ ]:
ls

'C3 - 05.12.2019.txt'   'F3 - 05.12.2019.txt'
'ECG - 05.12.2019.txt'  'O2 - 05.12.2019.txt'


## Storing data to csv file

In [ ]:
C3 = txt_to_arr('C3 - 05.12.2019.txt')
F3 = txt_to_arr('F3 - 05.12.2019.txt')
O2 = txt_to_arr('O2 - 05.12.2019.txt')
ECG = txt_to_arr('ECG - 05.12.2019.txt')
ECG = ECG[range(0, len(ECG), 2)]
len(C3), len(F3), len(O2), len(ECG)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Length of data :  6456126
Length of data :  6456126
Length of data :  6456126
Length of data :  12912251


(6456126, 6456126, 6456126, 6456126)

In [ ]:
df = pd.DataFrame()
df['C3'] = C3
df['F3'] = F3
df['O2'] = O2
df['ECG'] = ECG
df.to_csv('Durgesh_Singh_C3_F3_O2_ECG.csv', index = None)
df.head()

,C3,F3,O2,ECG
0,38.0,302.0,-5.0,479.0
1,1.0,-9.0,13.0,331.0
2,15.0,3.0,8.0,361.0
3,20.0,2.0,9.0,405.0
4,26.0,6.0,8.0,338.0


## Reading csv file

In [ ]:
df = pd.read_csv('Durgesh_Singh_C3_F3_O2_ECG.csv')
df.head()

,C3,F3,O2,ECG
0,38.0,302.0,-5.0,479.0
1,1.0,-9.0,13.0,331.0
2,15.0,3.0,8.0,361.0
3,20.0,2.0,9.0,405.0
4,26.0,6.0,8.0,338.0


In [ ]:
C3 = df.C3.values
F3 = df.F3.values
O2 = df.O2.values
ECG = df.ECG.values
len(C3), len(F3), len(O2), len(ECG)

(6456126, 6456126, 6456126, 6456126)

## Extracting and Storing Features from C3 channel

In [ ]:
C3 = C3/max(C3)
area_C3 = np.array([], dtype = float)
for i in range(int(len(C3)/7680)-1):
  #print(i)
  x = C3[i*7680:i*7680+7680]
  x1 = C3[i*7680:i*7680+2*7680]
  #print(x)
  ar = np.asarray([find_area(x,x1, dim = [dim], tau = tau)],dtype=float)
  area_C3 = np.append(area_C3,ar)
print(len(area_C3))
ar_label = pd.DataFrame({'area_C3':area_C3})
ar_label.head()

839


,area_C3
0,22.213548
1,1.042608
2,2.079507
3,6.114661
4,0.863315


<Figure size 432x288 with 0 Axes>

## Extracting and Features from F3 channel

In [ ]:
F3 = F3/max(F3)
area_F3 = np.array([], dtype = float)
for i in range(int(len(F3)/7680)-1):
  #print(i)
  x = F3[i*7680:i*7680+7680]
  x1 = F3[i*7680:i*7680+2*7680]
  #print(x)
  ar = np.asarray([find_area(x,x1, dim = [dim], tau = tau)],dtype=float)
  area_F3 = np.append(area_F3,ar)
print(len(area_F3))
ar_label['area_F3'] = area_F3
ar_label.head()

839


,area_C3,area_F3
0,22.213548,11.521899
1,1.042608,0.862500
2,2.079507,1.984668
3,6.114661,0.534293
4,0.863315,1.066610


<Figure size 432x288 with 0 Axes>

## Extracting and Storing Features from O2 Channel

In [ ]:
O2 = O2/max(O2)
area_O2 = np.array([], dtype = float)
for i in range(int(len(O2)/7680)-1):
  #print(i)
  x = O2[i*7680:i*7680+7680]
  x1 = O2[i*7680:i*7680+2*7680]
  #print(x)
  ar = np.asarray([find_area(x,x1, dim = [dim], tau = tau)],dtype=float)
  area_O2 = np.append(area_O2,ar)
print(len(area_O2))
ar_label['area_O2'] = area_O2
ar_label.head()

839


,area_C3,area_F3,area_O2
0,22.213548,11.521899,0.469130
1,1.042608,0.862500,0.424595
2,2.079507,1.984668,2.551703
3,6.114661,0.534293,2.477053
4,0.863315,1.066610,3.549870


<Figure size 432x288 with 0 Axes>

## Extracting and Storing Features from ECG Channel

In [ ]:
ECG = ECG/max(ECG)
area_ECG = np.array([], dtype = float)
for i in range(int(len(ECG)/7680)-1):
  #print(i)
  x = ECG[i*7680:i*7680+7680]
  x1 = ECG[i*7680:i*7680+2*7680]
  #print(x)
  ar = np.asarray([find_area(x,x1, dim = [dim], tau = tau)],dtype=float)
  area_ECG = np.append(area_ECG,ar)
print(len(area_ECG))
ar_label['area_ECG'] = area_ECG
ar_label.head()

839


,area_C3,area_F3,area_O2,area_ECG
0,22.213548,11.521899,0.469130,25.598502
1,1.042608,0.862500,0.424595,10.694522
2,2.079507,1.984668,2.551703,10.934460
3,6.114661,0.534293,2.477053,43.729325
4,0.863315,1.066610,3.549870,10.840449


<Figure size 432x288 with 0 Axes>

In [ ]:
ar_label.to_csv('Feature_C3_F3_O2_ECG_nolabel_Durgesh_constant_tau_dim.csv', index = False)
df = pd.read_csv('Feature_C3_F3_O2_ECG_nolabel_Durgesh_constant_tau_dim.csv')
df.head()

,area_C3,area_F3,area_O2,area_ECG
0,22.213548,11.521899,0.469130,25.598502
1,1.042608,0.862500,0.424595,10.694522
2,2.079507,1.984668,2.551703,10.934460
3,6.114661,0.534293,2.477053,43.729325
4,0.863315,1.066610,3.549870,10.840449


# Exctracting Features from Shobhit

In [ ]:
cd ../Shobhit \Gupta

/content/drive/My Drive/Sleep data/Shobhit Gupta


In [ ]:
ls

'C3 - 06.08.2019.txt'   'F3 - 06.08.2019.txt'
'ECG - 06.08.2019.txt'  'O2 - 06.08.2019.txt'


## Storing the Data in csv file

In [ ]:
C3 = txt_to_arr('C3 - 06.08.2019.txt')
F3 = txt_to_arr('F3 - 06.08.2019.txt')
O2 = txt_to_arr('O2 - 06.08.2019.txt')
ECG = txt_to_arr('ECG - 06.08.2019.txt')
ECG = ECG[range(0, len(ECG), 2)]
len(C3), len(F3), len(O2), len(ECG)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Length of data :  4619072
Length of data :  4619072
Length of data :  4619072
Length of data :  9238144


(4619072, 4619072, 4619072, 4619072)

In [ ]:
df = pd.DataFrame()
df['C3'] = C3
df['F3'] = F3
df['O2'] = O2
df['ECG'] = ECG
df.to_csv('Shobhit_Gupta_C3_F3_O2_ECG.csv', index = None)
df.head()

,C3,F3,O2,ECG
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0


## Reading csv file

In [ ]:
df = pd.read_csv('Shobhit_Gupta_C3_F3_O2_ECG.csv')
df.head()

,C3,F3,O2,ECG
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0


In [ ]:
C3 = df.C3.values
F3 = df.F3.values
O2 = df.O2.values
ECG = df.ECG.values
len(C3), len(F3), len(O2), len(ECG)

(4619072, 4619072, 4619072, 4619072)

## Extracting and Storing Features from C3 Channel

In [ ]:
C3 = C3/max(C3)
area_C3 = np.array([], dtype = float)
for i in range(int(len(C3)/7680)-1):
  #print(i)
  x = C3[i*7680:i*7680+7680]
  x1 = C3[i*7680:i*7680+2*7680]
  #print(x)
  ar = np.asarray([find_area(x,x1, dim = [dim], tau = tau)],dtype=float)
  area_C3 = np.append(area_C3,ar)
print(len(area_C3))
ar_label = pd.DataFrame({'area_C3':area_C3})
ar_label.head()

600


,area_C3
0,75.116188
1,5.244319
2,43.884235
3,55.355214
4,3.590647


<Figure size 432x288 with 0 Axes>

## Extracting and Storing Features from F3 Channel

In [ ]:
F3 = F3/max(F3)
area_F3 = np.array([], dtype = float)
for i in range(int(len(F3)/7680)-1):
  #print(i)
  x = F3[i*7680:i*7680+7680]
  x1 = F3[i*7680:i*7680+2*7680]
  #print(x)
  ar = np.asarray([find_area(x,x1, dim = [dim], tau = tau)],dtype=float)
  area_F3 = np.append(area_F3,ar)
print(len(area_F3))
ar_label['area_F3'] = area_F3
ar_label.head()

600


,area_C3,area_F3
0,75.116188,96.945072
1,5.244319,4.920165
2,43.884235,8.840105
3,55.355214,19.057999
4,3.590647,3.265240


<Figure size 432x288 with 0 Axes>

## Extracting and Storing Features from O2 Channel

In [ ]:
O2 = O2/max(O2)
area_O2 = np.array([], dtype = float)
for i in range(int(len(O2)/7680)-1):
  #print(i)
  x = O2[i*7680:i*7680+7680]
  x1 = O2[i*7680:i*7680+2*7680]
  #print(x)
  ar = np.asarray([find_area(x,x1, dim = [dim], tau = tau)],dtype=float)
  area_O2 = np.append(area_O2,ar)
print(len(area_O2))
ar_label['area_O2'] = area_O2
ar_label.head()

600


,area_C3,area_F3,area_O2
0,75.116188,96.945072,95.060392
1,5.244319,4.920165,13.587425
2,43.884235,8.840105,12.945789
3,55.355214,19.057999,5.506793
4,3.590647,3.265240,5.352508


<Figure size 432x288 with 0 Axes>

## Extracting and Storing Features from ECG Channel

In [ ]:
ECG = ECG/max(ECG)
area_ECG = np.array([], dtype = float)
for i in range(int(len(ECG)/7680)-1):
  #print(i)
  x = ECG[i*7680:i*7680+7680]
  x1 = ECG[i*7680:i*7680+2*7680]
  #print(x)
  ar = np.asarray([find_area(x,x1, dim = [dim], tau = tau)],dtype=float)
  area_ECG = np.append(area_ECG,ar)
print(len(area_ECG))
ar_label['area_ECG'] = area_ECG
ar_label.head()

600


,area_C3,area_F3,area_O2,area_ECG
0,75.116188,96.945072,95.060392,31.459880
1,5.244319,4.920165,13.587425,41.162437
2,43.884235,8.840105,12.945789,39.619453
3,55.355214,19.057999,5.506793,38.975962
4,3.590647,3.265240,5.352508,39.249151


<Figure size 432x288 with 0 Axes>

In [ ]:
ar_label.to_csv('Feature_C3_F3_O2_ECG_nolabel_Shobhit_constant_tau_dim.csv', index = False)
df = pd.read_csv('Feature_C3_F3_O2_ECG_nolabel_Shobhit_constant_tau_dim.csv')
df.head()

,area_C3,area_F3,area_O2,area_ECG
0,75.116188,96.945072,95.060392,31.459880
1,5.244319,4.920165,13.587425,41.162437
2,43.884235,8.840105,12.945789,39.619453
3,55.355214,19.057999,5.506793,38.975962
4,3.590647,3.265240,5.352508,39.249151


# Extracting Features from Subham

## Storing Data in csv file

In [ ]:
cd ../Subham\ Tripathi

/content/drive/My Drive/Sleep data/Subham Tripathi


In [ ]:
ls

'C3 - 06.08.2019.txt'   'F3 - 06.08.2019.txt'
'ECG - 06.08.2019.txt'  'O2 - 06.08.2019.txt'


In [ ]:
C3 = txt_to_arr('C3 - 06.08.2019.txt')
F3 = txt_to_arr('F3 - 06.08.2019.txt')
O2 = txt_to_arr('O2 - 06.08.2019.txt')
ECG = txt_to_arr('ECG - 06.08.2019.txt')
ECG = ECG[range(0, len(ECG), 2)]
len(C3), len(F3), len(O2), len(ECG)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Length of data :  4619072
Length of data :  4619072
Length of data :  4619072
Length of data :  9238144


(4619072, 4619072, 4619072, 4619072)

In [ ]:
df = pd.DataFrame()
df['C3'] = C3
df['F3'] = F3
df['O2'] = O2
df['ECG'] = ECG
df.to_csv('Subham_Tripathy_C3_F3_O2_ECG.csv', index = None)
df.head()

,C3,F3,O2,ECG
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0


## Reading csv file

In [ ]:
df = pd.read_csv('Subham_Tripathy_C3_F3_O2_ECG.csv')
df.head()

,C3,F3,O2,ECG
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0


In [ ]:
C3 = df.C3.values
F3 = df.F3.values
O2 = df.O2.values
ECG = df.ECG.values
len(C3), len(F3), len(O2), len(ECG)

(4619072, 4619072, 4619072, 4619072)

## Extracting and Storing Features from C3 channel

In [ ]:
C3 = C3/max(C3)
area_C3 = np.array([], dtype = float)
for i in range(int(len(C3)/7680)-1):
  #print(i)
  x = C3[i*7680:i*7680+7680]
  x1 = C3[i*7680:i*7680+2*7680]
  #print(x)
  ar = np.asarray([find_area(x,x1, dim = [dim], tau = tau)],dtype=float)
  area_C3 = np.append(area_C3,ar)
print(len(area_C3))
ar_label = pd.DataFrame({'area_C3':area_C3})
ar_label.head()

600


,area_C3
0,75.116188
1,5.244319
2,43.884235
3,55.355214
4,3.590647


<Figure size 432x288 with 0 Axes>

## Extracting and Storing Features from F3 Channel

In [ ]:
F3 = F3/max(F3)
area_F3 = np.array([], dtype = float)
for i in range(int(len(F3)/7680)-1):
  #print(i)
  x = F3[i*7680:i*7680+7680]
  x1 = F3[i*7680:i*7680+2*7680]
  #print(x)
  ar = np.asarray([find_area(x,x1, dim = [dim], tau = tau)],dtype=float)
  area_F3 = np.append(area_F3,ar)
print(len(area_F3))
ar_label['area_F3'] = area_F3
ar_label.head()

600


,area_C3,area_F3
0,75.116188,96.945072
1,5.244319,4.920165
2,43.884235,8.840105
3,55.355214,19.057999
4,3.590647,3.265240


<Figure size 432x288 with 0 Axes>

## Extracting and Storing Features from O2 Channel

In [ ]:
O2 = O2/max(O2)
area_O2 = np.array([], dtype = float)
for i in range(int(len(O2)/7680)-1):
  #print(i)
  x = O2[i*7680:i*7680+7680]
  x1 = O2[i*7680:i*7680+2*7680]
  #print(x)
  ar = np.asarray([find_area(x,x1, dim = [dim], tau = tau)],dtype=float)
  area_O2 = np.append(area_O2,ar)
print(len(area_O2))
ar_label['area_O2'] = area_O2
ar_label.head()

600


,area_C3,area_F3,area_O2
0,75.116188,96.945072,95.060392
1,5.244319,4.920165,13.587425
2,43.884235,8.840105,12.945789
3,55.355214,19.057999,5.506793
4,3.590647,3.265240,5.352508


<Figure size 432x288 with 0 Axes>

## Extracting and Storing Features from ECG Channel

In [ ]:
ECG = ECG/max(ECG)
area_ECG = np.array([], dtype = float)
for i in range(int(len(ECG)/7680)-1):
  #print(i)
  x = ECG[i*7680:i*7680+7680]
  x1 = ECG[i*7680:i*7680+2*7680]
  #print(x)
  ar = np.asarray([find_area(x,x1, dim = [dim], tau = tau)],dtype=float)
  area_ECG = np.append(area_ECG,ar)
print(len(area_ECG))
ar_label['area_ECG'] = area_ECG
ar_label.head()

600


,area_C3,area_F3,area_O2,area_ECG
0,75.116188,96.945072,95.060392,31.459880
1,5.244319,4.920165,13.587425,41.162437
2,43.884235,8.840105,12.945789,39.619453
3,55.355214,19.057999,5.506793,38.975962
4,3.590647,3.265240,5.352508,39.249151


<Figure size 432x288 with 0 Axes>

In [ ]:
ar_label.to_csv('Feature_C3_F3_O2_ECG_nolabel_Subham_constant_tau_dim.csv', index = False)
df = pd.read_csv('Feature_C3_F3_O2_ECG_nolabel_Subham_constant_tau_dim.csv')
df.head()

,area_C3,area_F3,area_O2,area_ECG
0,75.116188,96.945072,95.060392,31.459880
1,5.244319,4.920165,13.587425,41.162437
2,43.884235,8.840105,12.945789,39.619453
3,55.355214,19.057999,5.506793,38.975962
4,3.590647,3.265240,5.352508,39.249151


# Extracting Features from Sumit

In [ ]:
cd ../Sumit\ Rawat

/content/drive/My Drive/Sleep data/Sumit Rawat


In [ ]:
ls

'C3 - 06.08.2019.txt'      'E2 - 06.08.2019.txt'    'M1 - 06.08.2019.txt'
'C3M2 - 06.08.2019.txt'    'E2M1 - 06.08.2019.txt'  'M2 - 06.08.2019.txt'
'C4 - 06.08.2019.txt'      'E2M2 - 06.08.2019.txt'  'O1 - 06.08.2019.txt'
'C4M1 - 06.08.2019.txt'    'ECG - 06.08.2019.txt'   'O1M2 - 06.08.2019.txt'
'Chin 1 - 06.08.2019.txt'  'F3 - 06.08.2019.txt'    'O2 - 06.08.2019.txt'
'E1 - 06.08.2019.txt'      'F3M2 - 06.08.2019.txt'  'O2M1 - 06.08.2019.txt'
'E1M1 - 06.08.2019.txt'    'F4 - 06.08.2019.txt'    'Sum RIPs - 06.08.2019.txt'
'E1M2 - 06.08.2019.txt'    'F4M1 - 06.08.2019.txt'


## Storing Data in csv file

In [ ]:
C3 = txt_to_arr('C3 - 06.08.2019.txt')
F3 = txt_to_arr('F3 - 06.08.2019.txt')
O2 = txt_to_arr('O2 - 06.08.2019.txt')
ECG = txt_to_arr('ECG - 06.08.2019.txt')
ECG = ECG[range(0, len(ECG), 2)]
len(C3), len(F3), len(O2), len(ECG)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Length of data :  4619072
Length of data :  4619072
Length of data :  4619072
Length of data :  9238144


(4619072, 4619072, 4619072, 4619072)

In [ ]:
df = pd.DataFrame()
df['C3'] = C3
df['F3'] = F3
df['O2'] = O2
df['ECG'] = ECG
df.to_csv('Sumit_Rawat_C3_F3_O2_ECG.csv', index = None)
df.head()

,C3,F3,O2,ECG
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0


## Reading csv file

In [ ]:
df = pd.read_csv('Sumit_Rawat_C3_F3_O2_ECG.csv')
df.head()

,C3,F3,O2,ECG
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0


In [ ]:
C3 = df.C3.values
F3 = df.F3.values
O2 = df.O2.values
ECG = df.ECG.values
len(C3), len(F3), len(O2), len(ECG)

(4619072, 4619072, 4619072, 4619072)

## Extracting and Storing Features from C3 Channel

In [ ]:
C3 = C3/max(C3)
area_C3 = np.array([], dtype = float)
for i in range(int(len(C3)/7680)-1):
  #print(i)
  x = C3[i*7680:i*7680+7680]
  x1 = C3[i*7680:i*7680+2*7680]
  #print(x)
  ar = np.asarray([find_area(x,x1, dim = [dim], tau = tau)],dtype=float)
  area_C3 = np.append(area_C3,ar)
print(len(area_C3))
ar_label = pd.DataFrame({'area_C3':area_C3})
ar_label.head()

600


,area_C3
0,75.116188
1,5.244319
2,43.884235
3,55.355214
4,3.590647


<Figure size 432x288 with 0 Axes>

## Extracting and Storing Features from F3 Channel

In [ ]:
F3 = F3/max(F3)
area_F3 = np.array([], dtype = float)
for i in range(int(len(F3)/7680)-1):
  #print(i)
  x = F3[i*7680:i*7680+7680]
  x1 = F3[i*7680:i*7680+2*7680]
  #print(x)
  ar = np.asarray([find_area(x,x1, dim = [dim], tau = tau)],dtype=float)
  area_F3 = np.append(area_F3,ar)
print(len(area_F3))
ar_label['area_F3'] = area_F3
ar_label.head()

600


,area_C3,area_F3
0,75.116188,96.945072
1,5.244319,4.920165
2,43.884235,8.840105
3,55.355214,19.057999
4,3.590647,3.265240


<Figure size 432x288 with 0 Axes>

## Extracting and Storing Features from O2 Channel

In [ ]:
O2 = O2/max(O2)
area_O2 = np.array([], dtype = float)
for i in range(int(len(O2)/7680)-1):
  #print(i)
  x = O2[i*7680:i*7680+7680]
  x1 = O2[i*7680:i*7680+2*7680]
  #print(x)
  ar = np.asarray([find_area(x,x1, dim = [dim], tau = tau)],dtype=float)
  area_O2 = np.append(area_O2,ar)
print(len(area_O2))
ar_label['area_O2'] = area_O2
ar_label.head()

600


,area_C3,area_F3,area_O2
0,75.116188,96.945072,95.060392
1,5.244319,4.920165,13.587425
2,43.884235,8.840105,12.945789
3,55.355214,19.057999,5.506793
4,3.590647,3.265240,5.352508


<Figure size 432x288 with 0 Axes>

## Extracting and Storing Features from ECG Channel

In [ ]:
ECG = ECG/max(ECG)
area_ECG = np.array([], dtype = float)
for i in range(int(len(ECG)/7680)-1):
  #print(i)
  x = ECG[i*7680:i*7680+7680]
  x1 = ECG[i*7680:i*7680+2*7680]
  #print(x)
  ar = np.asarray([find_area(x,x1, dim = [dim], tau = tau)],dtype=float)
  area_ECG = np.append(area_ECG,ar)
print(len(area_ECG))
ar_label['area_ECG'] = area_ECG
ar_label.head()

600


,area_C3,area_F3,area_O2,area_ECG
0,75.116188,96.945072,95.060392,31.459880
1,5.244319,4.920165,13.587425,41.162437
2,43.884235,8.840105,12.945789,39.619453
3,55.355214,19.057999,5.506793,38.975962
4,3.590647,3.265240,5.352508,39.249151


<Figure size 432x288 with 0 Axes>

In [ ]:
ar_label.to_csv('Feature_C3_F3_O2_ECG_nolabel_Sumit_constant_tau_dim.csv', index = False)
df = pd.read_csv('Feature_C3_F3_O2_ECG_nolabel_Sumit_constant_tau_dim.csv')
df.head()

,area_C3,area_F3,area_O2,area_ECG
0,75.116188,96.945072,95.060392,31.459880
1,5.244319,4.920165,13.587425,41.162437
2,43.884235,8.840105,12.945789,39.619453
3,55.355214,19.057999,5.506793,38.975962
4,3.590647,3.265240,5.352508,39.249151


# Extracting Features from Sunny

In [ ]:
cd ../Sunny\ Thakur

/content/drive/My Drive/Sleep data/Sunny Thakur


In [ ]:
ls

'C3 - 01.07.2019.txt'   'F3 - 01.07.2019.txt'
'ECG - 01.07.2019.txt'  'O2 - 01.07.2019.txt'


## Storing Data in csv file

In [ ]:
C3 = txt_to_arr('C3 - 01.07.2019.txt')
F3 = txt_to_arr('F3 - 01.07.2019.txt')
O2 = txt_to_arr('O2 - 01.07.2019.txt')
ECG = txt_to_arr('ECG - 01.07.2019.txt')
ECG = ECG[range(0, len(ECG), 2)]
len(C3), len(F3), len(O2), len(ECG)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Length of data :  7032064
Length of data :  7032064
Length of data :  7032064
Length of data :  14064128


(7032064, 7032064, 7032064, 7032064)

In [ ]:
df = pd.DataFrame()
df['C3'] = C3
df['F3'] = F3
df['O2'] = O2
df['ECG'] = ECG
df.to_csv('Sunny_Thakur_C3_F3_O2_ECG.csv', index = None)
df.head()

,C3,F3,O2,ECG
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0


## Reading csv file

In [ ]:
df = pd.read_csv('Sunny_Thakur_C3_F3_O2_ECG.csv')
df.head()

,C3,F3,O2,ECG
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0


In [ ]:
C3 = df.C3.values
F3 = df.F3.values
O2 = df.O2.values
ECG = df.ECG.values
len(C3), len(F3), len(O2), len(ECG)

(7032064, 7032064, 7032064, 7032064)

## Extracting and Storing Features from C3 Channel

In [ ]:
C3 = C3/max(C3)
area_C3 = np.array([], dtype = float)
for i in range(int(len(C3)/7680)-1):
  #print(i)
  x = C3[i*7680:i*7680+7680]
  x1 = C3[i*7680:i*7680+2*7680]
  #print(x)
  ar = np.asarray([find_area(x,x1, dim = [dim], tau = tau)],dtype=float)
  area_C3 = np.append(area_C3,ar)
print(len(area_C3))
ar_label = pd.DataFrame({'area_C3':area_C3})
ar_label.head()

914


,area_C3
0,3.814876
1,2.849026
2,2.041976
3,2.466733
4,2.091364


<Figure size 432x288 with 0 Axes>

## Extracting and Storing Features from F3 Channel

In [ ]:
F3 = F3/max(F3)
area_F3 = np.array([], dtype = float)
for i in range(int(len(F3)/7680)-1):
  #print(i)
  x = F3[i*7680:i*7680+7680]
  x1 = F3[i*7680:i*7680+2*7680]
  #print(x)
  ar = np.asarray([find_area(x,x1, dim = [dim], tau = tau)],dtype=float)
  area_F3 = np.append(area_F3,ar)
print(len(area_F3))
ar_label['area_F3'] = area_F3
ar_label.head()

914


,area_C3,area_F3
0,3.814876,3.026601
1,2.849026,2.944856
2,2.041976,2.215776
3,2.466733,2.836901
4,2.091364,2.322230


<Figure size 432x288 with 0 Axes>

## Extracting and Storing Features from O2 Channel

In [ ]:
O2 = O2/max(O2)
area_O2 = np.array([], dtype = float)
for i in range(int(len(O2)/7680)-1):
  #print(i)
  x = O2[i*7680:i*7680+7680]
  x1 = O2[i*7680:i*7680+2*7680]
  #print(x)
  ar = np.asarray([find_area(x,x1, dim = [dim], tau = tau)],dtype=float)
  area_O2 = np.append(area_O2,ar)
print(len(area_O2))
ar_label['area_O2'] = area_O2
ar_label.head()

914


,area_C3,area_F3,area_O2
0,3.814876,3.026601,2.995074
1,2.849026,2.944856,3.102988
2,2.041976,2.215776,2.318145
3,2.466733,2.836901,2.795980
4,2.091364,2.322230,2.135181


<Figure size 432x288 with 0 Axes>

## Extracting and Storing Features from ECG Channel

In [ ]:
ECG = ECG/max(ECG)
area_ECG = np.array([], dtype = float)
for i in range(int(len(ECG)/7680)-1):
  #print(i)
  x = ECG[i*7680:i*7680+7680]
  x1 = ECG[i*7680:i*7680+2*7680]
  #print(x)
  ar = np.asarray([find_area(x,x1, dim = [dim], tau = tau)],dtype=float)
  area_ECG = np.append(area_ECG,ar)
print(len(area_ECG))
ar_label['area_ECG'] = area_ECG
ar_label.head()

914


,area_C3,area_F3,area_O2,area_ECG
0,3.814876,3.026601,2.995074,12.430815
1,2.849026,2.944856,3.102988,7.730752
2,2.041976,2.215776,2.318145,26.337589
3,2.466733,2.836901,2.795980,7.636212
4,2.091364,2.322230,2.135181,29.918835


<Figure size 432x288 with 0 Axes>

In [ ]:
ar_label.to_csv('Feature_C3_F3_O2_ECG_nolabel_Sunny_constant_tau_dim.csv', index = False)
df = pd.read_csv('Feature_C3_F3_O2_ECG_nolabel_Sunny_constant_tau_dim.csv')
df.head()

,area_C3,area_F3,area_O2,area_ECG
0,3.814876,3.026601,2.995074,12.430815
1,2.849026,2.944856,3.102988,7.730752
2,2.041976,2.215776,2.318145,26.337589
3,2.466733,2.836901,2.795980,7.636212
4,2.091364,2.322230,2.135181,29.918835
